# Manejo de datos ausentes

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos.
En particular, **a muchos conjuntos de datos interesantes les faltará cierta cantidad de datos**.
Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar datos faltantes de diferentes maneras.

En esta sección, discutiremos algunas consideraciones generales para los datos faltantes, discutiremos cómo Pandas elige representarlos y demostraremos algunas herramientas integradas de Pandas para manejar datos faltantes en Python.
**Aquí y a lo largo del libro, nos referiremos a los datos faltantes en general como valores *nulo*, *NaN* o *NA*.**

## Compensaciones en las convenciones sobre datos faltantes

Se han desarrollado varios esquemas para indicar la presencia de datos faltantes en una tabla o marco de datos.
Generalmente, giran en torno a una de dos estrategias: **usar una *máscara* que indica globalmente valores faltantes, o elegir un *valor centinela* que indica una entrada faltante**.

En el enfoque de enmascaramiento, **la máscara** podría ser una matriz booleana completamente separada, o puede implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque centinela, el valor centinela **podría ser alguna convención específica de datos, como indicar un valor entero faltante con -9999** o algún patrón de bits raro, o podría ser una convención más global, como indicar un valor centinela valor de punto flotante con **NaN (no es un número)**, un valor especial que forma parte de la especificación de punto flotante IEEE.

Ninguno de estos enfoques está exento de compensaciones: el uso de una matriz de máscara separada requiere la asignación de una matriz booleana adicional, lo que agrega gastos generales tanto en el almacenamiento como en el cálculo. Un valor centinela reduce el rango de valores válidos que se pueden representar y puede requerir lógica adicional (a menudo no optimizada) en la aritmética de CPU y GPU. Los valores especiales comunes como NaN no están disponibles para todos los tipos de datos.

Como ocurre en la mayoría de los casos en los que no existe una opción universalmente óptima, diferentes lenguajes y sistemas utilizan diferentes convenciones.
**Por ejemplo, el lenguaje R utiliza patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican datos faltantes**, mientras que el sistema SciDB utiliza un byte adicional adjunto a cada celda que indica un estado NA.

## Datos ausentes en pandas

La forma en que Pandas **maneja los valores faltantes está limitada por su dependencia del paquete NumPy**, que no tiene una noción incorporada de valores NA para tipos de datos que no son de punto flotante.

Pandas podría haber seguido **el ejemplo de R al especificar patrones de bits para cada tipo de datos individual** para indicar la nulidad, pero este enfoque resulta bastante difícil de manejar.
Si bien R contiene cuatro tipos de datos básicos, **NumPy admite *mucho* más que esto**: por ejemplo, mientras que R tiene un único tipo de entero, NumPy admite *catorce* tipos de enteros básicos una vez que se tienen en cuenta las precisiones disponibles, la firma y endianidad de la codificación.
**Reservar un patrón de bits específico en todos los tipos de NumPy disponibles generaría una cantidad excesiva de gastos generales** en varias operaciones de carcasas especiales para varios tipos, lo que probablemente incluso requeriría una nueva bifurcación del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para usarlo como máscara reducirá significativamente el rango de valores que puede representar.

**NumPy admite matrices enmascaradas**, es decir, matrices que tienen una matriz de máscara booleana adjunta adjunta para marcar datos como "buenos" o "malos".
Pandas podría haberse derivado de esto, pero la sobrecarga tanto en almacenamiento, computación y mantenimiento del código hace que sea una elección poco atractiva.

Con estas limitaciones en mente, **Pandas optó por usar centinelas para los datos ausentes** y, además, optó por usar dos valores nulos de Python ya existentes: el valor especial de punto flotante **``NaN`` y el valor de Python ``. Objeto 'None'``**.
Esta elección tiene algunos efectos secundarios, como veremos, pero en la práctica acaba siendo un buen compromiso en la mayoría de los casos de interés.

### ``None``: Pythonic missing data

El primer valor centinela utilizado por Pandas es **``None``**, un objeto singleton de Python que se utiliza a menudo para datos faltantes en el código Python.
Debido a que es un objeto Python, ``None`` no se puede usar en ningún arreglo NumPy/Pandas arbitrario, sino solo en arreglos con tipo de datos ``'objeto'`` (es decir, arreglos de objetos Python):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

Este ``dtype=object`` significa que la mejor representación de tipo común que NumPy podría inferir para el contenido de la matriz es que son objetos de Python.
Si bien este tipo de matriz de objetos es útil para algunos propósitos, cualquier operación con los datos se realizará en el nivel de Python, con mucha más sobrecarga que las operaciones típicamente rápidas que se ven para matrices con tipos nativos:

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
57 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.22 ms ± 78 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



El uso de objetos Python en una matriz también significa que si realiza agregaciones como ``sum()`` o ``min()`` en una matriz con un valor ``None``, generalmente obtendrá un error:

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

Esto refleja el hecho de que la suma entre un número entero y ``None`` no está definida.

### ``NaN``: Faltan datos numéricos

La otra representación de datos que falta, ``NaN`` (acrónimo de *Not a Number*), es diferente; es un valor especial de punto flotante reconocido por todos los sistemas que utilizan la representación de punto flotante estándar IEEE:

In [5]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

Observe que NumPy eligió un tipo de punto flotante nativo para esta matriz: esto significa que **a diferencia de la matriz de objetos anterior, esta matriz admite operaciones rápidas insertadas en código compilado.**
Debe tener en cuenta que ``NaN`` es un poco como un virus de datos: infecta cualquier otro objeto que toca.
Independientemente de la operación, el resultado de la aritmética con ``NaN`` será otro ``NaN``:

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

In [10]:
vals2 + 1

array([ 2., nan,  4.,  5.])

Tenga en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no dan lugar a un error), pero no siempre son útiles:

In [9]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy proporciona algunas agregaciones especiales que ignorarán estos valores faltantes:

In [11]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

In [12]:
np.nanmean(vals2)

2.6666666666666665

Tenga en cuenta que ``NaN`` es específicamente un valor de punto flotante; no existe un valor NaN equivalente para números enteros, cadenas u otros tipos.

### NaN y None en Pandas

``NaN`` y ``None`` tienen su lugar, y Pandas está diseñado para manejarlos dos casi indistintamente, realizando conversiones entre ellos cuando corresponda:

In [14]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Para los tipos que no tienen un valor centinela disponible, Pandas convierte automáticamente el tipo cuando hay valores NA presentes.
Por ejemplo, **si establecemos un valor en una matriz de enteros en ``np.nan``, se convertirá automáticamente a un tipo de punto flotante para acomodar el NA**:

In [15]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [16]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Tenga en cuenta que **además de convertir la matriz de enteros a punto flotante, Pandas convierte automáticamente ``Ninguno`` en un valor ``NaN``.**
(Tenga en cuenta que existe una propuesta para agregar un entero nativo NA a Pandas en el futuro; al momento de escribir este artículo, no se ha incluido).

Si bien este tipo de magia puede parecer un poco pirateada en comparación con el enfoque más unificado de los valores NA en lenguajes de dominios específicos como R, el enfoque centinela/casting de Pandas funciona bastante bien en la práctica y, en mi experiencia, rara vez causa problemas.

La siguiente tabla enumera las convenciones de actualización en Pandas cuando se introducen valores NA:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Tenga en cuenta que en Pandas, los datos de cadena siempre se almacenan con un tipo d ``objeto``.

## Operando con valores nulos

Como hemos visto, **Pandas trata ``None`` y ``NaN`` como esencialmente intercambiables para indicar valores nulos o faltantes.**
Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Ellos son:

- ``isnull()``: Genera una máscara booleana que indica los valores faltantes
- ``notnull()``: Opuesto de ``isnull()``
- ``dropna()``: Devuelve una versión filtrada de los datos
- ``fillna()``: Devuelve una copia de los datos con los valores faltantes completados o imputados

Concluiremos esta sección con una breve exploración y demostración de estas rutinas.

### Detectando valores nulos
Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull()`` y ``notnull()``.
Cualquiera de los dos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [32]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [33]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [34]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

Como se menciona en la parte de Data Indexing and Selection, las máscaras booleanas se pueden utilizar directamente como índice ``Series`` o ``DataFrame``:

In [20]:
data[data.notnull()] #filtrado, me selecciona los Trues

0        1
2    hello
dtype: object

In [21]:
data[data.isnull()]

1     NaN
3    None
dtype: object

In [35]:
data[data==1]

0    1
dtype: object

In [22]:
data[~(data==1)] #~ filtrado de lo opuesto que esta entre parentesis, es muy util para filtrados

1      NaN
2    hello
3     None
dtype: object

Los métodos ``isnull()`` y ``notnull()`` producen resultados booleanos similares para ``DataFrames``.

### Eliminando valores nulos

Además del enmascaramiento utilizado anteriormente, existen métodos convenientes, ``dropna()``
(que elimina los valores de NA) y ``fillna()`` (que completa los valores de NA). Para una ``Serie``,
el resultado es sencillo:

In [36]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [37]:
# data.dropna()
data.dropna(inplace=True) # Hace lo mismo
data = data.dropna() # Hace lo mismo

In [38]:
data

0        1
2    hello
dtype: object

Para un ``DataFrame``, hay más opciones.
Considere el siguiente ``DataFrame``:

In [53]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [54]:
df.info() #numero de filas o index son los registros pero las columnas son las variables, por eso info analiza las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2 non-null      float64
 1   1       2 non-null      float64
 2   2       3 non-null      int64  
dtypes: float64(2), int64(1)
memory usage: 204.0 bytes


No podemos eliminar valores individuales de un ``DataFrame``; solo podemos eliminar filas o columnas completas.
Dependiendo de la aplicación, es posible que desee uno u otro, por lo que ``dropna()`` ofrece varias opciones para un ``DataFrame``.

De forma predeterminada, ``dropna()`` eliminará todas las filas en las que *cualquier* valor nulo esté presente:

In [55]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternativamente, puede colocar los valores NA a lo largo de un eje diferente; ``axis=1`` elimina todas las columnas que contienen un valor nulo:

In [56]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [57]:
df.dropna(axis=1, inplace=True) #por defecto lo hace por filas, si quiero hacerlo por columnas modifico el valor de axis
# df.dropna(axis='columns')

In [58]:
df

,2
0,2
1,5
2,6


Pero esto también arroja algunos buenos datos; Es posible que prefiera eliminar filas o columnas con *todos* los valores de NA, o la mayoría de los valores de NA.
Esto se puede especificar a través de los parámetros ``how`` o ``thresh``, que permiten un control preciso del número de valores nulos que se permiten.

El valor predeterminado es ``how='any'``, de modo que cualquier fila o columna (dependiendo de la palabra clave ``axis``) que contenga un valor nulo se eliminará.
También puede especificar ``how='all'``, lo que solo eliminará filas/columnas que sean *todos* valores nulos:

In [59]:
df[3] = np.nan
df

,2,3
0,2,NaN
1,5,NaN
2,6,NaN


In [60]:
df.dropna(axis='columns', how='all') #how = 'any' elimina la columna entera si hay al menos un NaN en ella

,2
0,2
1,5
2,6


Para un control más detallado, el parámetro ``thresh`` le permite especificar un número mínimo de valores no nulos para la fila/columna que se mantendrá:

In [61]:
df[2]

0    2
1    5
2    6
Name: 2, dtype: int64

In [62]:
df.dropna(axis='rows', thresh=3)

,2,3


In [63]:
len(df.columns)*0.75

1.5

In [64]:
df

,2,3
0,2,NaN
1,5,NaN
2,6,NaN


In [65]:
df.dropna(axis='rows', thresh=len(df.columns)*0.75) #muy util para filtrar: parametro subset

,2,3


In [66]:
len(df)*0.75

2.25

In [67]:
df.dropna(axis='columns', thresh=len(df)*0.75)

,2
0,2
1,5
2,6


Aquí, la primera y la última fila se han eliminado porque contienen solo dos valores no nulos.

### Llenando valores nulos

A veces, en lugar de eliminar los valores de NA, es mejor reemplazarlos con un valor válido.
Este valor podría ser un número único como cero, o podría ser algún tipo de imputación o interpolación de los buenos valores.
Podrías hacer esto in situ usando el método ``isnull()`` como máscara, pero debido a que es una operación tan común, Pandas proporciona el método ``fillna()``, que devuelve una copia de la matriz con el valores nulos reemplazados.

Considere la siguiente ``Serie``:

In [68]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Podemos completar las entradas de NA con un solo valor, como cero:

In [69]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Podemos especificar un relleno hacia adelante para propagar el valor anterior hacia adelante:

In [70]:
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [71]:
# forward-fill
data.fillna(method='ffill')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16220\3316037056.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill')


a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

O podemos especificar un relleno para propagar los siguientes valores hacia atrás:

In [72]:
# back-fill
data.fillna(method='bfill')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16220\692461307.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill')


a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Para los ``DataFrames``, las opciones son similares, pero también podemos especificar un ``eje`` a lo largo del cual se realizan los rellenos:

In [73]:
df

,2,3
0,2,NaN
1,5,NaN
2,6,NaN


In [74]:
df.fillna(df[2].mean()) #es mejor sustitur por la mediana que por la media, en este codigo sustituyo los NaN por la media de la variable 2

,2,3
0,2,4.333333
1,5,4.333333
2,6,4.333333


In [83]:
df

,2,3
0,2,NaN
1,5,NaN
2,6,NaN


In [76]:
df.fillna(method='bfill', axis=1)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16220\1498511816.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', axis=1)


,2,3
0,2.0,NaN
1,5.0,NaN
2,6.0,NaN


Tenga en cuenta que si un valor anterior no está disponible durante un llenado directo, el valor NA permanece.